# XML Parser -- Sedgwick

In [1]:
# Import necessary libraries.
import re, glob, csv, sys, os
import pandas as pd
import xml.etree.ElementTree as ET

# Declare directory location to shorten filepaths later.
abs_dir = "/Users/quinn.wi/Documents/"

# Gather all .xml files using glob.
list_of_files = glob.glob(abs_dir + "Data/PSC/Sedgwick/*.xml")

## Functions

In [2]:
'''
Arguments of Functions:

    namespace:

    ancestor:
    
    xpath_as_string:
    
    attrib_val_str:
    
'''

# Read in file and get root of XML tree.
def get_root(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    return root


# Get namespace of individual file from root element.
def get_namespace(root):
    namespace = re.match(r"{(.*)}", str(root.tag))
    ns = {"ns":namespace.group(1)}
    return ns

## Parser Files

In [3]:
%%time

dataframe = []

for file in list_of_files:
    
    try:
        root = get_root(file)
        ns = get_namespace(root)

        reFile = str(re.search(r'.*/(.*.xml)', str(file)).group(1)) # get filename without path

        date = root.find('.//ns:date/[@type="creation"]', ns).get('when') # get date.

        source = root.find('.//ns:author', ns).text   # get source/author & target/recipient
        target = root.find('.//ns:recipient', ns).text

    #     Loops
    #     loop to get all references (persRef)
        references_l = []
        for ref in root.findall('.//ns:persRef', ns):
            person = ref.get('ref')
            references_l.append(person)
        references = ','.join(references_l)

    #     loop to get subjects.
        subject_l = []
        for subject in root.findall('.//ns:subject', ns):
            subject_l.append(subject.text)
        subjects = ','.join(subject_l)

    #     loop to get all text within <div type="docbody">
        text_l = []
        for txt in root.findall('.//ns:div[@type="docbody"]', ns):
            string = ''.join(ET.tostring(txt, encoding='unicode', method='text'))
            clean_string = re.sub(r'[\t\n\s]+', ' ', string)
            text_l.append(clean_string)
        content = ' '.join(text_l)


        row = {'file': reFile, 'data': date, 'source': source, 'target':target, 
               'subjects': subjects, 'references': references, 'text': content}
        

        dataframe.append(row)
        
    except:
        print (file, '\n')
    
df = pd.DataFrame(dataframe)

df.head(3)

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1807-04-26-toFrancesSedgwickWatsonFD.xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1803-10-06-toPamelaDwightSedgwickF.xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1809-01-27-toTheodoreSedgwickIFD.xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1807-12-25-toFrancesSedgwickWatsonFD.xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1806-01-17-toPamelaDwightSedgwickFD (1).xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1805-11-29-toPamelaDwightSedgwickFD.xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1807-04-26-toFSWF.xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1800-01-12-toTheodoreSedgwickIF.xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1805-11-15-toPamelaDwightSedgwickFD (1).xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1807-12-28-toFrancesSedgwickWatsonFD.xml 

/Users/quinn.wi/Documents/Data/PSC/Sedgwick/CMS1807-03-24-toFrancesSedgwickWatsonFD.xml 

/Users/quinn.wi/Documents/Data/PSC/

,file,data,source,target,subjects,references,text
0,CMS1819-03-08-toRobertSedgwickIF (1).xml,1819-03-08,Catharine Maria Sedgwick,sedgwick-robert,,"sedgwick-charles,sedgwick-elizabeth,sedgwick-h...",Albany March 8' 1819 -- I came here my dear Ro...
1,CMS1816-03-25-toFrancesSedgwickWatsonF.xml,1816-03-25,Catharine Maria Sedgwick,FSW,,"RSI,banyer-maria,jay-sarah,van vechten-jacob,s...",Albany March 25th 1816 I have just heard of an...
2,CMS1813-08-15-toRobertSedgwickIF.xml,1813-08-15,Catharine Maria Sedgwick,RSI,,"FSW,U,payne-eloise,warner-thomas,warner-france...",Stockbridge August 15th 1813 I recollect very...


In [4]:
%%time

df.to_csv(abs_dir + 'Data/Output/ParsedXML/Sedgwick_dataframe.txt', sep='\t', index=False)

CPU times: user 12.4 ms, sys: 2.44 ms, total: 14.8 ms
Wall time: 14.6 ms
